In [ ]:
__author__ = "Theodora Chu, Josh Cohen, Jason Chen"
__version__ = "CS224u, Stanford, Spring 2016 term"

# Setup

In [4]:
import os
import nltk

# File Input
Takes in a text file and returns a list of ordered unigrams U. 
It should also consider stemming and other relevant pre-processing. Josh's note: parse "African American" as a unigram.

# Correlation Matrix
1. Parse U to create a word-word frequency matrix M, where each row represents a word and each entry x(i,j) represents the number of times word i co-occurs with word j.
2. Convert M to a new matrix M’ with some sort of correlation operation. We could use PMI, Occai (see Josh’s paper), CSA, or some other correlation structure.
3. Let row a represent the unigram “African American”. Take in that row, and output an ordered list of (this_unigram, correlation_score) pairs which represent the correlation score of this_unigram with the term “African American”
4. Produce a list L of the top 100 correlated words with the term “African American”


# Sentiment Analysis
Takes in a list V of words and returns the average sentiment score across all terms in V as determined by freebase. Note to Jason: consider other sentiment databases

In [6]:
def getSentiment(word):
    #score = nltk.sentiment.vader.polarity_scores(word)
    #replace this with score from freebase
    return len(word)

V = ['good', 'bad', 'great', 'worrisome', 'stupid']
def generate_sentiment(wordList):
    totalSentiment = 0.0;
    for word in wordList:
        totalSentiment += getSentiment(word)
    averageSentiment = totalSentiment/len(wordList)
    return averageSentiment

print generate_sentiment(V)

5.4


# XOR/AND
Takes in a dict of corpus:list of words and returns a dict of corpus:XOR words and dict of corpus:AND words.

In [8]:
def XOR(corpusLists):
    return 0
def AND(corpusLists):
    for list in corpusLists:
        for word in list:
            return 0
            #add to AND

# Word Cloud
Takes in a matrix M and correlation list L. Using t-sne, produces a word cloud which represents correlation between all terms. 